In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPooling2D
from keras.models import Sequential
import os


Using TensorFlow backend.


In [2]:
num_classes=6
img_rows,img_cols=48,48
batch_size=16

In [3]:
train_data='./fer2013/train'
validation_data='./fer2013/validation'

In [4]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=30,shear_range=0.3,zoom_range=0.3,width_shift_range=0.4,height_shift_range=0.4,
                                 horizontal_flip=True,fill_mode='nearest')
validation_datagen=ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_directory(train_data,color_mode='grayscale',target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,class_mode='categorical',shuffle=True)
validation_generator=validation_datagen.flow_from_directory(validation_data,color_mode='grayscale',target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,class_mode='categorical',shuffle=True)

Found 28273 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.


In [5]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer="he_normal",
                input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer="he_normal",
                input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64,kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes,kernel_initializer="he_normal"))
model.add(Activation('softmax'))

print(model.summary())



          
          
          





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)       

In [8]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [12]:
checkpoint=ModelCheckpoint("C:/Users/chinm/PycharmProjects/Emotion Detector/emotion_detect_save.h5",
                          monitor="val_loss",mode="min",
                          save_best_only=True,verbose=1)
earlystop=EarlyStopping(monitor="val_loss",min_delta=0,
                       patience=5,verbose=1,
                        restore_best_weights=True)
reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,
                           verbose=1,min_delta=0.0001)
callbacks=[earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
             metrics=['accuracy'])

nb_train_samples=28273
nb_validation_samples=3534
epochs=10

history=model.fit_generator(train_generator,
                            steps_per_epoch=nb_train_samples//batch_size,
                           epochs=epochs,
                           callbacks=callbacks,
                           validation_data=validation_generator,
                           validation_steps=nb_validation_samples//batch_size)

Epoch 1/10
1767/1767 [==============================] - 617s 349ms/step - loss: 1.7643 - accuracy: 0.2459 - val_loss: 1.8301 - val_accuracy: 0.2489

Epoch 00001: val_loss improved from inf to 1.83011, saving model to C:/Users/chinm/PycharmProjects/Emotion Detector/emotion_detect_save.h5
Epoch 2/10
1767/1767 [==============================] - 579s 328ms/step - loss: 1.7505 - accuracy: 0.2490 - val_loss: 1.8275 - val_accuracy: 0.2035

Epoch 00002: val_loss improved from 1.83011 to 1.82749, saving model to C:/Users/chinm/PycharmProjects/Emotion Detector/emotion_detect_save.h5
Epoch 3/10
1767/1767 [==============================] - 799s 452ms/step - loss: 1.7498 - accuracy: 0.2487 - val_loss: 1.6801 - val_accuracy: 0.2695

Epoch 00003: val_loss improved from 1.82749 to 1.68009, saving model to C:/Users/chinm/PycharmProjects/Emotion Detector/emotion_detect_save.h5
Epoch 4/10
1767/1767 [==============================] - 578s 327ms/step - loss: 1.7457 - accuracy: 0.2516 - val_loss: 1.6091 - v

In [13]:
from keras.models import load_model
classifier=load_model("C:/Users/chinm/PycharmProjects/Emotion Detector/emotion_detect_save.h5")

In [14]:
from keras.preprocessing import image
import numpy as np
from os import listdir
import cv2
from os.path import isfile,join
import re

In [15]:
validation_generator=validation_datagen.flow_from_directory(validation_data,color_mode='grayscale',target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,class_mode='categorical',shuffle=True)
class_labels=validation_generator.class_indices
class_labels={v:k for k ,v in class_labels.items()}
classes=list(class_labels.values())

Found 3534 images belonging to 6 classes.


In [48]:
def draw_test(name,pred,im,true_label):
    BLACK=[0,0,0]
    expanded_image=cv2.copyMakeBorder(im,160,0,0,300,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image,"predicted-"+pred,(20,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.putText(expanded_image, "true-" + true_label, (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,  255,0), 2)
    cv2.imshow(name,expanded_image)

In [49]:
def getRandomImage(path,img_width,img_height):
    folders=list(filter(lambda x:os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory=np.random.randint(0,len(folders))
    path_class=folders[random_directory]
    file_path=path+path_class
    file_names=[f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index=np.random.randint(0,len(file_names))
    image_name=file_names[random_file_index]
    final_path=file_path+"/"+image_name
    return image.load_img(final_path,target_size=(img_width,img_height),grayscale=True),final_path,path_class

In [50]:
img_width,img_height=48,48
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])
files=[]
predictions=[]
true_labels=[]

for i in range(0,10):
    path='./fer2013/validation/'
    img,final_path,true_label=getRandomImage(path,img_width,img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x=image.img_to_array(img)
    x=x*1./255
    x=np.expand_dims(x,axis=0)
    images=np.vstack([x])
    classes=model.predict_classes(images,batch_size=10)
    predictions.append(classes)

for i in range(0,len(files)):
    image=cv2.imread((files[i]))
    image=cv2.resize(image,None,fx=3,fy=3,interpolation=cv2.INTER_CUBIC)
    draw_test("Prediction",class_labels[predictions[i][0]],image,true_labels[i])
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [16]:
face_classifier=cv2.CascadeClassifier('C:/Users/chinm/AppData/Local/Programs/Python/Python37-32/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')
def face_detector(img):
    gray=cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    if faces is():
        return (0,0,0,0),np.zeros((48,48),np.uint8),img
    
    for (x,y,w,h) in faces:
        x=x-50
        y=y-50
        w=w+50
        h=h+50
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray=gray[y:y+h,x:x+w]
        try:
            roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        except:
            return (x,w,y,h),np.zeros((48,48),np.uint8),img
    return (x,w,y,h),roi_gray,img
    
    

In [17]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    rects,face,img=face_detector(frame)
    
    if np.sum([face])!=0.0:
        roi=face.astype("float")/255.0
        roi=keras.preprocessing.image.img_to_array(roi)
        roi=np.expand_dims(roi,axis=0)
        
        preds=classifier.predict(roi)[0]
        labels=class_labels[preds.argmax()]
        label_position=(rects[0]+int((rects[1]/2)),rects[2]+25)
        cv2.putText(img,labels,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow("Emotion Detector",img)
    if(cv2.waitKey(1)==13):
        break
cap.release()
cv2.destroyAllWindows()
    